In [ ]:
import os
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false" # 关闭部分 XLA 行为
os.environ["TF_CUDNN_DETERMINISTIC"] = "1" # 强制确定性算法，减少搜索

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import KFold
import itertools


#### Data Preparation and Preprocessing
load and normalize images

In [ ]:
import tensorflow as tf

# Define dataset paths
train_dir = "/home/zyh/Fruit-Classifier/data/train_augment"    # directory with subfolders for each class (train set)


IMG_SIZE = 224   # image size (can be adjusted as needed)



model structure

In [ ]:
from tensorflow.keras import layers, models, regularizers

def create_cnn_model():

    model = models.Sequential()

    #Convolution blocks
    model.add(layers.Conv2D(32, kernel_size = (3,3), 
                    padding='same',
                    input_shape=(IMG_SIZE, IMG_SIZE, 3),
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2)) 

    model.add(layers.Conv2D(64, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2)) 

    model.add(layers.Conv2D(128, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Conv2D(256, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Conv2D(512, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Conv2D(1024, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Conv2D(2048, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(4,activation='softmax'))


    #model.summary()

    
    
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Wrap the model creation in KerasClassifier for use in GridSearchCV
def build_model(learning_rate=0.01, optimizer_name='Adam', activation='relu', 
                dropout_rate=0.2, l2_rate=0.0):
    # Build the CNN model with given hyperparams
    model = create_cnn_model()
    # Choose optimizer
    if optimizer_name == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:  # 'SGD'
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'], jit_compile=False)
    return model


# 全内存数据加载函数 (速度优化的关键)
def load_data_to_memory(data_dir, img_size):
    print(f"正在加载数据到内存: {data_dir} ...")
    X = []
    y = []
    # 确保类名顺序一致: ['apple', 'banana', 'mixed', 'orange']
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    print(f"检测到的类别: {classes}")
    
    for label_idx, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        files = os.listdir(class_dir)
        for fname in files:
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                fpath = os.path.join(class_dir, fname)
                # 加载并调整大小
                img = load_img(fpath, target_size=(img_size, img_size))
                # 转为数组并归一化 (0-1)
                img_array = img_to_array(img) / 255.0
                
                X.append(img_array)
                y.append(label_idx)
                
    X = np.array(X)
    # 将标签转为 One-hot 编码 (e.g., [0, 1, 0, 0])
    y = tf.keras.utils.to_categorical(np.array(y), num_classes=len(classes))
    print(f"数据加载完成! X shape: {X.shape}, y shape: {y.shape}")
    return X, y



# 一次性读取所有数据到 RAM (320张图约占用 150MB~200MB 内存，非常安全)
X_all, y_all = load_data_to_memory(train_dir, IMG_SIZE)


#  极速版 Grid Search
param_grid = {
    'learning_rate': [0.01, 0.001,0.0005],
    'optimizer_name': ['SGD', 'Adam'],
    'batch_size': [16, 32],
    #'activation': ['relu'],
    'dropout_rate': [0.0,0.2],
    #'l2_rate': [0.0, 0.001]
}

# 使用5折交叉验证
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

best_acc = 0
best_params = None

# 生成所有组合
combinations = list(itertools.product(*param_grid.values()))
total_runs = len(combinations) * 5
current_run = 0

print(f"开始 Grid Search，总共要训练 {len(combinations)} 个模型配置...")

for combo in combinations:
    params = dict(zip(param_grid.keys(), combo))
    
    # 从 params 中分离出 batch_size，因为它不传给 build_model
    build_args = {k: v for k, v in params.items() if k != 'batch_size'}
    current_batch_size = params['batch_size']
    
    acc_list = []
    
    # K-Fold 循环
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
        # 清理旧模型，释放显存 
        tf.keras.backend.clear_session()
        
        # 划分数据 (直接内存切片，速度极快)
        X_train, X_val = X_all[train_idx], X_all[val_idx]
        y_train, y_val = y_all[train_idx], y_all[val_idx]
        
        # 构建模型
        model = build_model(**build_args)
        
        # 回调函数
        callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='val_loss'),    #先不保留权重
            # 网格搜索时 ReduceLROnPlateau 可能拖慢速度，简单起见可以先注释掉，或者保留
            # tf.keras.callbacks.ReduceLROnPlateau(patience=2) 
        ]
        
        # 训练 (无需 Generator，直接传数组)
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=current_batch_size,
            epochs=15, 
            callbacks=callbacks,
            verbose=0  # 静默模式，只打印结果
        )
        
        # 评估
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        acc_list.append(val_acc)
        
        current_run += 1
        # 打印进度条
        print(f"Run {current_run}/{total_runs} - Acc: {val_acc:.4f}")

    avg_acc = np.mean(acc_list)
    print(f"Params: {params} | Avg Acc: {avg_acc:.4f}")
    
    if avg_acc > best_acc:
        best_acc = avg_acc
        best_params = params
        print(f"🔥🔥🔥 发现新最佳精度: {best_acc:.4f}")

print("\n========================================")
print("搜索结束")
print(f"最佳精度: {best_acc}")
print(f"最佳参数: {best_params}")

A few notes on the above:

We defined build_model to accept the hyperparams. If activation='leaky_relu', inside create_cnn_model we would handle that by setting layers with no activation and adding LeakyReLU layers. (This implementation detail can be handled with an if inside create_cnn_model.)


------------------------------------------------------

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# grid_search.fit(X_train, y_train, **{'callbacks': [early_stop], 'validation_split': 0.1})


-------------------------------------------------------

#### Model Training with Best Hyperparameters and Early Stopping

In [ ]:
# # Build final model with best hyperparams
# best_model = create_cnn_model(activation=best_params['activation'], 
#                               dropout_rate=best_params['dropout_rate'], 
#                               l2_rate=best_params['l2_rate'])
# if best_params['optimizer_name'] == 'Adam':
#     final_optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
# else:
#     final_optimizer = tf.keras.optimizers.SGD(learning_rate=best_params['learning_rate'])
# best_model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Setup callbacks for early stopping (and optional learning rate reduction, checkpoints as in original code)
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
#     ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
#     ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
# ]

# # Train the model with early stopping
# history = best_model.fit(train_generator, epochs=50,  # start with an upper bound, early stopping will likely stop earlier
#                          validation_data=val_generator, 
#                          callbacks=callbacks, verbose=1)


Results and Visualization of Training Progress

In [ ]:
# val_loss, val_acc = best_model.evaluate(val_generator)
# print(f"Validation Accuracy: {val_acc:.2%}, Validation Loss: {val_loss:.4f}")


In [ ]:
# import matplotlib.pyplot as plt

# # Extract history data
# epochs = range(1, len(history.history['loss'])+1)
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']
# train_acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

# # Plot Loss Curves
# plt.figure(figsize=(6,4))
# plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
# plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
# plt.title('Training vs Validation Loss')
# plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.show()

# # Plot Accuracy Curves
# plt.figure(figsize=(6,4))
# plt.plot(epochs, train_acc, 'bo-', label='Training Accuracy')
# plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
# plt.title('Training vs Validation Accuracy')
# plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.show()
